In [16]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [17]:
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc

In [18]:
num_folds = 5

In [19]:
def non_discriminative(num_latent):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(num_folds=num_folds, fold_num=cur_fold)
        train_sc, test_sc = reshape_for_sc(train), reshape_for_sc(test)
        train_data = np.array([train_sc[:, :, i ] for i in range(1, train.shape[1])]).swapaxes(1, 2)
        c = SparseCoding()
        c.train(train_data, num_latent=num_latent)
        pred = c.disaggregate(test_sc[:, :, 0].swapaxes(0, 1)).swapaxes(0, 2).swapaxes(1, 2)
        pred = pred.reshape(pred.shape[0], pred.shape[1], -1, 24)
        out.append(pred)
    return np.concatenate(out)


In [32]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err_non_disc ={}
gt = tensor[:, 1:, :, :]
for num_latent in range(1, 30):
    print(num_latent)
    pred = non_discriminative(num_latent)
    # Clamping prediction to aggregate
    pred = np.minimum(pred, tensor[:, 0:1, :,:])
    err_non_disc[num_latent] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [36]:
pd.DataFrame(err_non_disc)[pd.DataFrame(err_non_disc).mean().argmin()]

dr         95.422585
dw         46.748713
fridge     75.369633
hvac      250.846110
mw         54.778683
Name: 26, dtype: float64